## Imports

In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from utils import *

In [11]:
orders, nodes = load_data("train_data/orders.csv", "train_data/nodes.csv")
X, y = feat_eng(orders, nodes)

In [12]:
X

,running_time,route_distance_km,latest10,distance,speed,expected_node_time,node_part_time,node_part_distance,time_rbf
Id,,,,,,,,,
1232212655921391683,30,8.023,1.000000,8.071906,7.222222,1.117649,0.001751,0.001006,0.031318
1232212655921391683,30,8.023,1.000000,54.631245,8.888889,6.146015,0.009631,0.006809,0.031371
1232212655921391683,30,8.023,1.000000,85.583772,14.166667,6.041207,0.009467,0.010667,0.031424
1232212655921391683,30,8.023,1.000000,74.058342,5.000000,14.811668,0.023211,0.009231,0.031552
1232212655921391683,30,8.023,1.000000,3.174041,13.333333,0.238053,0.000373,0.000396,0.031554
...,...,...,...,...,...,...,...,...,...
4378890784988522255,1435,4.739,1.309539,80.225910,10.000000,8.022591,0.027163,0.016929,0.405027
4378890784988522255,1435,4.739,1.309539,71.665261,9.722222,7.371284,0.024958,0.015122,0.404795
4378890784988522255,1435,4.739,1.309539,154.010596,9.166667,16.801156,0.056886,0.032499,0.404267


In [3]:
X_train, y_train, X_val, y_val = train_test_split(X, y)

In [4]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [5]:
rf.feature_importances_

array([0.01445326, 0.03640042, 0.01273929, 0.00547303, 0.00451899,
       0.67127151, 0.23253796, 0.01112802, 0.01147753])

In [6]:
y_pred_node = rf.predict(X_val)
mean_squared_error(y_pred_node, y_val, squared=False)

2.1062912024219442

In [7]:
y_pred_node = pd.Series(y_pred_node, index=y_val.index)
y_pred_total = get_eta_from_nodes(y_pred_node)
y_test_total = get_eta_from_nodes(y_val)
mean_squared_error(y_pred_total, y_test_total, squared=False)

87.15147805984927

In [33]:
orders_test, nodes_test = load_data("test_data/final_test.csv", "test_data/nodes_test.csv", False)
X, y = feat_eng(orders_test, nodes_test, train_orders=orders, train=False)

        running_time  route_distance_km  latest10    distance      speed  \
Id                                                                         
6198             218              4.744  1.000000  138.795710  10.277778   
6198             218              4.744  1.000000   95.273001   9.166667   
6198             218              4.744  1.000000  137.647881   9.722222   
6198             218              4.744  1.000000    4.383708   8.333333   
6198             218              4.744  1.000000   24.195593   8.611111   
...              ...                ...       ...         ...        ...   
527520          1132              0.703  0.949619  126.478060   6.666667   
527520          1132              0.703  0.949619   90.911869  10.833333   
527520          1132              0.703  0.949619   16.870887  12.500000   
527520          1132              0.703  0.949619   44.055661  12.500000   
527520          1132              0.703  0.949619   22.109122   6.944444   

        exp

In [34]:
y

Int64Index([  6198,   6198,   6198,   6198,   6198,   6198,   6198,   6198,
              6198,   6198,
            ...
            527520, 527520, 527520, 527520, 527520, 527520, 527520, 527520,
            527520, 527520],
           dtype='int64', name='Id', length=102496)

In [35]:
pred_nodes = rf.predict(X)
pred_nodes = pd.Series(pred_nodes, index=y)
pred = get_eta_from_nodes(pred_nodes)

pred

Id
6198      476.939408
6417      695.056060
7054      460.645619
9628      682.938070
10283     650.357200
             ...    
525706    450.311885
526604    539.362743
527213    509.980963
527520    215.648658
527850    755.677683
Length: 1000, dtype: float64

In [37]:
pred.to_csv("submission.csv")